In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import re

from datetime import datetime
import numpy as np

import warnings
warnings.filterwarnings('ignore')

# 지수 표기법을 10진수로 변경
pd.set_option('display.float_format', '{:.2f}'.format)

import platform

system = platform.system()

if system == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
    
else:
    !apt-get -y install fonts-nanum
    !fc-cache -fv
    !rm ~/.cache/matplotlib -rf

    plt.rcParams['font.family'] = 'NanumBarunGothic'

# minus 기호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

# 데이터 확인

In [2]:
df_reviews = pd.read_csv('./data/myfipn_db_reviews.csv')
df_blog = pd.read_csv('./data/myfipn_db_blog.csv')

In [5]:
df_reviews = df_reviews.drop(columns='_id').drop_duplicates()
df_blog = df_blog.drop(columns='_id').drop_duplicates()

In [13]:
# reviews의 date는 'YYYY-MM-DD' 형식의 문자열
df_reviews['date'] = pd.to_datetime(df_reviews['date'])

# blog의 date는 'YYYYMMDD' 형식의 정수
df_blog['date'] = pd.to_datetime(df_blog['date'].astype(str), format='%Y%m%d')

# 두 데이터프레임에 month 컬럼 생성 (YYYY-MM 형식)
df_reviews['month'] = df_reviews['date'].dt.strftime('%Y-%m')
df_blog['month'] = df_blog['date'].dt.strftime('%Y-%m')

In [15]:
df_reviews

,product,price,writer,content,date,rating,month
0,픽싱 • 라운 그란데 - 애쉬 13.5 [원먼스/2PCS],30000,김**,"NEW 고객님께서 해당 제품을 ""만족""으로 평가해주셨습니다.",2025-02-08,5.00,2025-02
2,픽싱 • 라운 그란데 - 애쉬 13.5 [원먼스/2PCS],30000,황**,"NEW 고객님께서 해당 제품을 ""만족""으로 평가해주셨습니다.",2025-02-08,5.00,2025-02
4,픽싱 • 라운 그란데 - 애쉬 13.5 [원먼스/2PCS],30000,김**,"NEW 고객님께서 해당 제품을 ""만족""으로 평가해주셨습니다.",2025-02-07,5.00,2025-02
5,픽싱 • 라운 그란데 - 애쉬 13.5 [원먼스/2PCS],30000,구**,"NEW 고객님께서 해당 제품을 ""만족""으로 평가해주셨습니다.",2025-02-07,5.00,2025-02
7,픽싱 • 라운 그란데 - 애쉬 13.5 [원먼스/2PCS],30000,이**,"NEW 고객님께서 해당 제품을 ""만족""으로 평가해주셨습니다.",2025-02-07,5.00,2025-02
...,...,...,...,...,...,...,...
5676,라운 그란데 - 베이지 13.5 [원데이/10PCS],23000,박**,"고객님께서 해당 제품을 ""아쉬움""으로 평가해주셨습니다.",2024-09-30,1.00,2024-09
5677,라운 그란데 - 베이지 13.5 [원데이/10PCS],23000,신**,"고객님께서 해당 제품을 ""보통""으로 평가해주셨습니다.",2024-09-29,3.00,2024-09
5678,라운 그란데 - 베이지 13.5 [원데이/10PCS],23000,김**,"고객님께서 해당 제품을 ""만족""으로 평가해주셨습니다.",2024-09-26,5.00,2024-09
5679,라운 그란데 - 베이지 13.5 [원데이/10PCS],23000,문**,"고객님께서 해당 제품을 ""만족""으로 평가해주셨습니다.",2024-09-24,5.00,2024-09
